In [1]:
from flask import Flask, request, redirect, url_for, jsonify
import pandas as pd
import os
import atexit
app = Flask(__name__)
import csv
import re
from nltk.tokenize import word_tokenize
import nltk
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Melvina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Melvina\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
app = Flask(__name__)

data_file = 'data.csv'

if not os.path.exists(data_file):
    df = pd.DataFrame(columns=['Name', 'Age', 'Major'])
    df.to_csv(data_file, index=False)

def load_data():
    df = pd.read_csv(data_file)
    return df.to_dict(orient='records')

def save_data(data):
    df = pd.DataFrame(data)
    df.to_csv(data_file, index=False)

data = load_data()

def close_app():
    save_data(data)
    print("Closing the app gracefully...")
    exit(0)

atexit.register(close_app)

@app.route('/')
def index():
    records_html = ''
    return f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
        <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.16.0/umd/popper.min.js"></script>
        <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>

        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>University Database</title>
        
        <!-- Bootstrap CSS CDN -->
        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
        
        <!-- Custom CSS -->
        <style>
            body {{
                background-color: #f8f9fa;
            }}
            .container {{
                padding-top: 50px;
            }}
            .card {{
                background-color: #ffffff;
                border-radius: 10px;
                box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            }}
        </style>
    </head>
    <body>
    
    <div class="container">
        <div class="row justify-content-center">
            <div class="col-md-10">
                <div class="card">
                    <div class="card-body">
                        <h2 class="card-title text-center">University Database</h2>
                        <form action="/add_student" method="post">
                            <div class="form-group">
                                <label for="name">Name</label>
                                <input type="text" class="form-control" id="name" name="name" required>
                            </div>
                            <div class="form-group">
                                <label for="age">Age</label>
                                <input type="number" class="form-control" id="age" name="age" required>
                            </div>
                            <div class="form-group">
                                <label for="major">Major</label>
                                <input type="text" class="form-control" id="major" name="major" required>
                            </div>
                            <button type="submit" class="btn btn-primary btn-block">Add Student</button>
                        </form>
                        <button class="btn btn-danger mt-4 ml-4 mb-4" onclick="window.location.href='/close_app'">Close App</button>
                        <button class="btn btn-danger mt-4 ml-4 mb-4" onclick="window.location.href='/view_records'">View Records</button>
                        
                        <!-- You can link the delete and update to appropriate forms where user inputs ID and other info -->
                        <button class="btn btn-secondary mt-4 ml-4 mb-4" onclick="window.location.href='/delete_form'">Delete Records</button>
                        <button class="btn btn-danger mt-4 ml-4 mb-4" onclick="window.location.href='/update_form'">Update Records</button>
                        <button onclick="window.location.href='/search_form'" class="btn btn-info mt-4 ml-4 mb-4">Search</button>

                    </div>
                </div>
            </div>
        </div>
    </div>
    

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">    
    <!-- Bootstrap JS (optional) -->
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>

    
    </body>
    </html>
    """


@app.route('/add_student', methods=['POST'])
def add_student():
    global data
    name = request.form['name']
    age = int(request.form['age'])
    major = request.form['major']
    data.append({'Name': name, 'Age': age, 'Major': major})
    save_data(data)
    return redirect(url_for('index'))

@app.route('/delete_student', methods=['POST'])
def delete_student():
    global data
    name_to_delete = request.form['name_to_delete']
    data = [record for record in data if record['Name'] != name_to_delete]
    save_data(data)
    return redirect(url_for('index'))

@app.route('/update_student', methods=['POST'])
def update_student():
    global data
    name_to_update = request.form['name_to_update']
    new_name = request.form['new_name']
    new_age = request.form['new_age']
    new_major = request.form['new_major']
    for record in data:
        if record['Name'] == name_to_update:
            record['Name'] = new_name
            record['Age'] = new_age
            record['Major'] = new_major
            break
    save_data(data)
    return redirect(url_for('index'))

@app.route('/delete_form')
def delete_form():
    return f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Delete Student</title>
        <!-- Bootstrap CSS -->
        <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
        <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.16.0/umd/popper.min.js"></script>
        <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
    </head>
    <body>
        <div class="container mt-5">
            <h2 class="text-center">Delete Student</h2>
            <form action="/delete_student" method="post">
                <div class="form-group">
                    <label for="name_to_delete">Name of student to delete</label>
                    <input type="text" class="form-control" id="name_to_delete" name="name_to_delete" required>
                </div>
                <button type="submit" class="btn btn-secondary btn-block">Delete Student</button>
            </form>
        </div>
    </body>
    </html>
    """

@app.route('/update_form')
def update_form():
    return f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Update Student</title>
        <!-- Bootstrap CSS -->
        <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
        <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.16.0/umd/popper.min.js"></script>
        <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
    </head>
    <body>
        <div class="container mt-5">
            <h2 class="text-center">Update Student</h2>
            <form action="/update_student" method="post">
                <div class="form-group">
                    <label for="name_to_update">Name of student to update</label>
                    <input type="text" class="form-control" id="name_to_update" name="name_to_update" required>
                </div>
                <!-- Add fields for the new student data -->
                <div class="form-group">
                    <label for="new_name">New Name</label>
                    <input type="text" class="form-control" id="new_name" name="new_name">
                </div>
                <div class="form-group">
                    <label for="new_age">New Age</label>
                    <input type="number" class="form-control" id="new_age" name="new_age">
                </div>
                <div class="form-group">
                    <label for="new_major">New Major</label>
                    <input type="text" class="form-control" id="new_major" name="new_major">
                </div>
                <button type="submit" class="btn btn-danger btn-block">Update Student</button>
            </form>
        </div>
    </body>
    </html>
    """



@app.route('/view_records', methods=['GET'])
def view_records():
    data_subset = pd.read_csv(data_file).head(20)  # Display only the first 20 records
    
    table_html = data_subset.to_html(classes='table table-striped table-bordered')
    
    # Build the final HTML
    html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>

        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Student Records</title>
        <!-- Bootstrap CSS CDN -->
        <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
    </head>
    <body>
        <div class="container mt-5">
            {table_html}
        </div>
    </body>
    </html>
    """

    return html_content



@app.route('/search_records', methods=['POST'])
def search_records():
    try:
        query = request.form['search_query']
        criteria = interpret_query_with_nltk(query)
        
        print(f"Interpreted criteria: {criteria}")  # Debugging print

        results = []
        with open("data.csv", "r") as file:
            reader = csv.DictReader(file)
            for row in reader:
                if match_query(row, criteria):
                    results.append(row)
        
        print(f"Results found: {len(results)}")  # Debugging print
        
        html_results = convert_results_to_html(results)
        return html_results

    except Exception as e:
        print(f"Error: {e}")  # This will print the error to your console
        return f"Error occurred: {e}"  # This will display the error on the webpage

@app.route('/search_form')
def search_form():
    return f"""
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    
    <div class="container">
        <h2 class="text-center">Search Students</h2>
        <form action="/search_records" method="post">
            <div class="form-group">
                <label for="search_query">Enter your query:</label>
                <input type="text" class="form-control" id="search_query" name="search_query" required>
            </div>
            <button type="submit" class="btn btn-info btn-block">Search</button>
        </form>
    </div>
    """


def interpret_query_with_nltk(query):
    tokens = [t.lower() for t in word_tokenize(query)]
    print("Tokens:", tokens)  # Debugging

    # Named Entity Recognition to extract names
    entities = pos_tag(tokens)
    names = [word for word, pos in entities if pos in ['NNP', 'NNPS']]
    print("Names detected:", names)  # Debugging

    criteria = {"name": None, "age_range": None, "major": None}

    # Age patterns
    age_pattern_range = re.compile(r'(\d{1,2})\s*-\s*(\d{1,2})')
    age_pattern_between = re.compile(r'between\s*(\d{1,2})\s*and\s*(\d{1,2})')
    
    age_match_range = age_pattern_range.search(query)
    age_match_between = age_pattern_between.search(query)

    # For age
    if age_match_range:
        criteria["age_range"] = tuple(map(int, age_match_range.groups()))
    elif age_match_between:
        criteria["age_range"] = tuple(map(int, age_match_between.groups()))
    print("Age Range:", criteria["age_range"])  # Debugging

    # For name
    name_keywords = ["tell me major of", "name", "of", "list", "down", "the names", "ask", "mention"]
    if "tell me major of" in query.lower():
        name_index = tokens.index("of") + 1 if "of" in tokens else None
        if name_index and name_index < len(tokens):
            criteria["name"] = tokens[name_index]
    elif any(word in tokens for word in name_keywords) and not criteria.get("age_range"):
        criteria["name"] = ' '.join(names)

    # Improved Major Extraction
    major_pattern = re.compile(r'majors? in (\w+(?: \w+)*)')
    major_match = major_pattern.search(query)
    if major_match:
        criteria["major"] = major_match.group(1)
    print("Major:", criteria["major"])  # Debugging

    return criteria


    
def match_query(row, criteria):
    name_match = True
    age_match = True
    major_match = True

    # Check for name
    if criteria["name"]:
        name_match = criteria["name"].lower() in row["Name"].lower()

    # Check for age
    if criteria["age_range"]:
        age = int(row["Age"])
        min_age, max_age = criteria["age_range"]
        age_match = min_age <= age <= max_age

    # Check for major
    if criteria["major"]:
        major_match = criteria["major"].lower() in row["Major"].lower()

    return name_match and age_match and major_match

def search_csv(criteria):
    results = []
    with open('data.csv', 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if match_query(row, criteria):
                results.append(row)
    return results

def convert_results_to_html(results):
    # Bootstrap CSS link
    bootstrap_css = '<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">'
    
    # Starting the HTML document
    html = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Search Results</title>
        {bootstrap_css}
    </head>
    <body>
    <div class="container mt-5">
    """

    # If no results found
    if not results:
        html += "<p>No results found.</p>"
    else:
        # Building the Bootstrap table
        html += "<table class='table table-striped table-bordered'><thead class='thead-dark'><tr><th>Name</th><th>Age</th><th>Major</th></tr></thead><tbody>"
        for result in results:
            html += f"<tr><td>{result['Name']}</td><td>{result['Age']}</td><td>{result['Major']}</td></tr>"
        html += "</tbody></table>"

    # Closing the HTML tags
    html += """
    </div>
    </body>
    </html>
    """
    
    return html

@app.route('/close_app')
def close_app_route():  
    os._exit(0)

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)

 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Dec/2023 20:01:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2023 20:01:53] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Dec/2023 20:01:56] "GET /view_records HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2023 20:03:22] "GET /delete_form HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2023 20:04:20] "GET /update_form HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2023 20:05:37] "GET /search_form HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2023 20:05:41] "POST /search_records HTTP/1.1" 200 -


Tokens: ['list', 'all', 'records', 'of', 'aged', 'between', '16', 'and', '22']
Names detected: []
Age Range: (16, 22)
Major: None
Interpreted criteria: {'name': None, 'age_range': (16, 22), 'major': None}
Results found: 7
